# Exploring Pandas with NYC Flights Dataset

R users will recognize this famous
dataset as a favorite of Hadley Wickham, author of dplyr.  No matter, we can use
it with pandas as
well.

This dataset will give us flight delay data from all
flights departing from the three NYC airports (JFK, LGA, EWR) in the year 2013.

## Step-1: Get Data

In [ ]:
# get data
import os
import urllib.request

data_location = "../data/nycflights13/flights.csv.gz"
#data_url = 'https://elephantscale-public.s3.amazonaws.com/data/nycflights13/flights.csv.gz'
data_url = 'https://github.com/elephantscale/datasets/blob/master/nycflights13/flights.csv.gz?raw=true'

if not os.path.exists (data_location):
    data_location = os.path.basename(data_location)
    if not os.path.exists(data_location):
        urllib.request.urlretrieve(data_url, data_location)
        print ('Downloading : ', data_url)
print('data_location :', data_location)  

In [ ]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

pd.options.display.float_format = '{:,.2f}'.format
flights = pd.read_csv(data_location)
flights

## Step-2: Calculate Arrival Delay

Let us focus on some summary and visualizations that we can do here. Let's start
out by finding the average delay by month.  We'll also do a matlibplot on it.

In [ ]:
# first let's extract 'arr_delay' column
flights['arr_delay']

In [ ]:
# then let's calculate average / mean
flights['arr_delay'].mean()

In [ ]:
## Now that we have done that, let's group by month and do the same

mean_delay_by_month = flights.groupby(['month'])['arr_delay'].mean()
mean_delay_by_month

In [ ]:
## Plot this

mean_delay_by_month.plot(kind='bar', title="Mean Delay By Month")


Notice that flights in two months actually have negative delay.  No doubt
airlines "pad" flight times to achieve better results.

## Step-3: Airport Specific Delays

One would guess that
flights to certain airports may have different results. For example, flights to
Chicago (ORD) would no doubt be affected by winter weather. Let's take a look at
that.

In [ ]:
## First step, extract all flights to Chicago (ORD)

flights[flights.dest == 'ORD']

In [ ]:
## Now do the same calculation

mean_delay_by_month_ord = flights[(flights.dest == 'ORD')].groupby(['month'])['arr_delay'].mean()
mean_delay_by_month_ord


In [ ]:
mean_delay_by_month_ord.plot(kind='bar', title="Mean Delay By Month (Chicago)")


In [ ]:
# Let's try Los Angeles and compare that

mean_delay_by_month_lax = flights[(flights.dest == 'LAX')].groupby(['month'])['arr_delay'].mean()
print("Flights to Los Angeles (LAX)")
print(mean_delay_by_month_lax)

mean_delay_by_month_lax.plot(kind='bar', title="Mean Delay By Month (Los Angeles)")


## Step-4: Delay by Airline

Perhaps we would like to see which Airline carrier is the best for delays.  Here
we have some plots to do that.

In [ ]:
# Examine if specific carrier will create different delay impact
arr_delay_by_carrier = flights.groupby('carrier')['arr_delay'].mean()
print (arr_delay_by_carrier)

# plot this
arr_delay_by_carrier.plot(kind='bar')
plt.xticks(rotation=0)
plt.xlabel('Carrier')
plt.ylabel('Average Delay in Min')
plt.title('Average Arrival Delay by Carrier in 2008, All  airports')

We see that F9 (Frontier Airlines) is among the worst in terms of delays, while
Hawaiian (HA) is among the best.

## Step-5: Joins

We have a couple of other tables in our dataset: Weather, and
airports. Let's look at these two tables and see if we can perform joins.

In [ ]:
import os
import urllib.request

data_location = "../data/nycflights13/weather.csv.gz"
data_url = 'https://elephantscale-public.s3.amazonaws.com/data/nycflights13/weather.csv.gz'

if not os.path.exists (data_location):
    data_location = os.path.basename(data_location)
    if not os.path.exists(data_location):
        urllib.request.urlretrieve(data_url, data_location)
        print ('Downloading : ', data_url)
print('data_location :', data_location)  

weather = pd.read_csv(data_location)
weather

In [ ]:
flights_withweather = pd.merge(flights, weather, how='left', on=['year','month', 'day', 'hour', 'origin'])
flights_withweather

In [ ]:
import os
import urllib.request

data_location = "../data/nycflights13/airports.csv.gz"
data_url = 'https://elephantscale-public.s3.amazonaws.com/data/nycflights13/airports.csv.gz'

if not os.path.exists (data_location):
    data_location = os.path.basename(data_location)
    if not os.path.exists(data_location):
        urllib.request.urlretrieve(data_url, data_location)
        print ('Downloading : ', data_url)
print('data_location :', data_location)  

airports = pd.read_csv(data_location)
airports

In [ ]:
flights_withairport = pd.merge(flights_withweather, airports, how='left', left_on='dest', right_on='faa')
flights_withairport